In [ ]:
import os

# load all datasets in synergy_dataset
path = 'synergy_dataset'
files = os.listdir(path)
files = [f for f in files if f.endswith('.csv')]

In [ ]:
# settings generator 

settings = []
exclude = ["nb:tfidf","nn-2-layer:tfidf","rf:reuseable_MiniLM","logistic:tfidf"]

for model in ["logistic"]:                          # "nb","logistic", "nn-2-layer", "xgboost", "rf", "svm"
    for fe in ["tfidf"]:                            # "tfidf", "doc2vec", "embedding-idf", "reuseable_MiniLM", "reuseable_sbert"
#       for b in ["max"]:                           # "simple", "double", "triple", "undersample"
#           for q in ["double"]:                    # "max", "max_random", "max_uncertainty", "uncertainty", "cluster"
                if not f"{model}:{fe}" in exclude:
                    settings.append("-m {model} -e {fe}".format(model=model, fe=fe))
print(settings, "\n",settings.__len__()*26*109)

In [ ]:
for i, setting in enumerate(settings):
    i = i+2
    for dataset in files:
        template = f"""apiVersion: batch/v1
kind: Job
metadata:
  name: sim-{dataset.lower().replace("_","-")[:-4]}-{i}
spec:
  ttlSecondsAfterFinished: 100
  template:
      metadata:
        labels:
          app: simulation
      spec:
        restartPolicy: Never
        containers:
        - name: sim
          image: ghcr.io/jteijema/asreview-simulation-project:latest
          resources:
            requests:
              memory: 4Gi
              cpu: 500m
          env:
          - name: DATASET
            value: {dataset[:-4]}
          - name: SETTINGS
            value: "{setting}"
          - name: EXOSCALE_API_KEY
            valueFrom:
              secretKeyRef:
                name: exoscale-credentials
                key: EXOSCALE_API_KEY
          - name: EXOSCALE_API_SECRET
            valueFrom:
              secretKeyRef:
                name: exoscale-credentials
                key: EXOSCALE_API_SECRET
          - name: EXOSCALE_STORAGE_ZONE
            valueFrom:
              secretKeyRef:
                name: exoscale-credentials
                key: EXOSCALE_STORAGE_ZONE
          - name: BUCKET
            valueFrom:
              secretKeyRef:
                name: exoscale-credentials
                key: BUCKET"""

        # save template to file
        with open(f"jobs/sim-{i}-{dataset.lower().replace('_','-')[:-4]}.yaml", "w") as f:
            f.write(template)
    

In [ ]:
# get all jobs
jobs = os.listdir('jobs')
for job in jobs:
    print(f"!kubectl create -f ./jobs/{job}")
    !kubectl create -f ./jobs/{job}